In [32]:
testdata = """Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3"""



In [33]:
#use an enum for gridvalues
import re

class Sensor:
    def __init__(self,sensorPos:tuple[int,int],beaconPos:tuple[int,int]):
        self.sensorPos = sensorPos
        self.beaconPos = beaconPos
        self.manhattanDistance = abs(beaconPos[0] - sensorPos[0]) + abs(beaconPos[1] - sensorPos[1])

    def __str__(self):
        return str(self.sensorPos) +':' + str(self.manhattanDistance)

    def filledRangeInRow(self,y:int)->tuple[int,int]:
        #return a range of x positions that represent where we know the cave isn't unknown
        dy = abs(y-self.sensorPos[1])
        if dy > self.manhattanDistance:
            return None
        else:
            xwidth = self.manhattanDistance - dy
            x = self.sensorPos[0]
            return (x-xwidth,x+xwidth)
    

        



class Cave:
    def __init__(self, input:str):
        self.sensors = []
        #parse the input
        pattern = '.*x=(-*[0-9]*).*y=(-*[0-9]*).*closest beacon is at x=(-*[0-9]*).*y=(-*[0-9]*)'
        for l in input.splitlines():
            match = re.search(pattern, l)
            sensorPos = (int(match.group(1)),int(match.group(2)))
            beaconPos = (int(match.group(3)),int(match.group(4)))
            self.sensors.append(Sensor(sensorPos,beaconPos))
    
    def knownPositionCountInRow(self, y:int)->int:
        #Need to loop through sensors
        #Will get a set of ranges back, that need to collapse into distinct overlapping ranges. Can then count each distinct range
        ranges:list[tuple[int,int]] = []
        distinctRanges:list[tuple[int,int]] = []

        for sensor in self.sensors:
            range = sensor.filledRangeInRow(y)
            print('Sensor ' + str(sensor) + ' --Range: ' +str(range))
            if range != None:
                ranges.append(range)
        
        #we have risk (present in test data) that our distinct range are no longer distict - a new range might have bridged them.
        #We could solve this if we sort the ranges first, by the left hand value
        ranges.sort(key=lambda x: x[0])
        print('--process ranges--')
        for left, right in ranges:
            print('range: '+str((left,right)))
            overlapFound = False
            for i, (rl, rr) in enumerate(distinctRanges):
                #rl, rr = distinctRanges[i]
                if not overlapFound:
                    #is this overlapping?
                    if (left <= rl and right >= rl) or (right >= rr and left <= rr) or (left >= rl and right <= rr):
                        #extend this range
                        distinctRanges[i] = (min(left,rl),max(right,rr))
                        print('Extending range: '+ str(rl)+':'+str(rr) + ' -> '+str(distinctRanges[i][0])+':'+str(distinctRanges[i][1]))
                        overlapFound = True
            if not overlapFound:
                print('New range: '+ str((left,right)))
                distinctRanges.append((left,right))
        
        


 
        #now count the size of each range
        count = 0
        print('--')
        for rl, rr in distinctRanges:
            print('Distinct Range: ' + str((rl,rr)))
            count += (rr - rl)
        return count





#tests
testCave = Cave(testdata)
print(len(testCave.sensors))
print(testCave.sensors[0])
print('----')
testResult = testCave.knownPositionCountInRow(10)
print(testResult)
print(testResult==26)



14
(2, 18):7
----
Sensor (2, 18):7 --Range: None
Sensor (9, 16):1 --Range: None
Sensor (13, 2):3 --Range: None
Sensor (12, 14):4 --Range: (12, 12)
Sensor (10, 20):4 --Range: None
Sensor (14, 17):5 --Range: None
Sensor (8, 7):9 --Range: (2, 14)
Sensor (2, 0):10 --Range: (2, 2)
Sensor (0, 11):3 --Range: (-2, 2)
Sensor (20, 14):8 --Range: (16, 24)
Sensor (17, 20):6 --Range: None
Sensor (16, 7):5 --Range: (14, 18)
Sensor (14, 3):1 --Range: None
Sensor (20, 1):7 --Range: None
--process ranges--
range: (-2, 2)
New range: (-2, 2)
range: (2, 14)
Extending range: -2:2 -> -2:14
range: (2, 2)
Extending range: -2:14 -> -2:14
range: (12, 12)
Extending range: -2:14 -> -2:14
range: (14, 18)
Extending range: -2:14 -> -2:18
range: (16, 24)
Extending range: -2:18 -> -2:24
--
Distinct Range: (-2, 24)
26
True


In [34]:
# using real puzzle input
input = open('day15input.txt').read()
cave = Cave(input)
result = cave.knownPositionCountInRow(2000000)
print(result)

Sensor (3859432, 2304903):1018240 --Range: (3146095, 4572769)
Sensor (2488890, 2695345):582168 --Range: None
Sensor (3901948, 701878):527376 --Range: None
Sensor (2422190, 1775708):881446 --Range: (1765036, 3079344)
Sensor (2703846, 3282799):635274 --Range: None
Sensor (172003, 2579074):867905 --Range: (-116828, 460834)
Sensor (1813149, 1311283):736830 --Range: (1765036, 1861262)
Sensor (1704453, 2468117):429497 --Range: None
Sensor (1927725, 2976002):315786 --Range: None
Sensor (3176646, 1254463):1142944 --Range: (2779239, 3574053)
Sensor (2149510, 3722117):520256 --Range: None
Sensor (3804434, 251015):408059 --Range: None
Sensor (2613561, 3932220):1194410 --Range: None
Sensor (3997794, 3291220):470809 --Range: None
Sensor (98328, 3675176):653862 --Range: None
Sensor (2006541, 2259601):479431 --Range: (1786711, 2226371)
Sensor (663904, 122919):1510811 --Range: None
Sensor (1116472, 3349728):1124023 --Range: None
Sensor (2810797, 2300748):269190 --Range: None
Sensor (1760767, 2024355):